In [ ]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from scipy.signal import medfilt

In [ ]:
input_dir = './images'
filename = 'trixie.jpg'
output_folder = './stencils'

In [ ]:
def make_stencil(image, num_colours):
    # Convert the image to greyscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Flatten the image into shape (-1, 1)
    flattened_image = np.reshape(gray_image, newshape=(-1, 1))
    
    # Cluster the image into num_colours distinct colours
    kmeans = KMeans(n_clusters=num_colours, random_state=0).fit(flattened_image)
    kmeans.cluster_centers_
    
    # Recolour the image using the most different shades possible
    vfunc = np.vectorize(lambda index: int(kmeans.cluster_centers_[index]))
    
    # Map the recolouring function onto the image
    _image = vfunc(np.reshape(kmeans.labels_, newshape=gray_image.shape))
    
    return _image

In [ ]:
# Read in the image in grayscale
image = cv2.imread(os.path.join(input_dir, filename))

name, extension = filename.split('.')

for num_colours in range(2, 10):
    # Make it into a stencil
    stencil = make_stencil(image, num_colours)

    # stencil = medfilt(stencil, 3)
    stencil.astype(np.uint8)

    if not os.path.exists(os.path.join(output_folder, name)):
        os.makedirs(os.path.join(output_folder, name))
        
    # Save the final image
    cv2.imwrite(
        os.path.join(output_folder, name, name + '_' + str(num_colours) + '.' + extension), 
        stencil
    )
